https://mp.weixin.qq.com/s/d8L5-bKxfQcW8B1lEj80NQ

### pytorch创建模型的四种方法

In [1]:
# 导入包
import torch
import torch.nn.functional as F
from collections import OrderedDict

#### 1.自定义类型[定义在init,前向过程在forward]

In [3]:
class Net1(torch.nn.Module):
    def __init__(self):
        super(Net1,self).__init__()
        self.conv1 = torch.nn.Conv2d(3,32,3,1,1)
        self.dense1 = torch.nn.Linear(32*3*3,128)
        self.dense2 = torch.nn.Linear(128,10)
    def forward(self,x):
        x = F.max_pool2d(F.relu(self.conv1(x)),2)
        x = x.view(x.size(x),-1)
        x = F.relu(self.dense1(x))
        x = self.dense2(x)
        
        return x

#### 2.序列继承型[利用nn.Sequential(顺序执行的层函数)]

In [4]:
class Net2(torch.nn.Module):
    def __init__(self):
        super(Net2,self).__init__()
        self.conv = torch.nn.Sequential(
            torch.nn.Conv2d(3,32,3,1,1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2)
        )
        self.dense = torch.nn.Sequential(
            torch.nn.Linear(32*3*3,128),
            torch.nn.ReLU(),
            torch.nn.Linear(128,10)
        )
    def forward(self,x):
        conv_out = self.conv(x)
        res = conv_out.view(conv_out.size(0),-1)
        out = self.dense(res)
        return out

#### 3.序列添加型[利用Sequential类add_module顺序逐层添加] \n
给予各层的name属性

In [5]:
class Net3(torch.nn.Module):
    def __init__(self):
        super(Net3,self).__init__()
        self.conv = torch.nn.Sequential()
        self.conv.add_module("conv1",torch.nn.Conv2d(3, 32, 3, 1, 1))
        self.conv.add_module("relu1",torch.nn.ReLU())
        self.conv.add_module("pool1",torch.nn.MaxPool2d(2))
        self.dense = torch.nn.Sequential()
        self.dense.add_module("dense1",torch.nn.Linear(32 * 3 * 3, 128))
        self.dense.add_module("relu2",torch.nn.ReLU())
        self.dense.add_module("dense2",torch.nn.Linear(128, 10))
    def forward(self,x):
        conv_out = self.conv(x)
        res = conv_out.view(conv_out.size(0), -1)
        out = self.dense(res)
        return out

#### 4.序列集成字典型[OrderDict集成模型字典['name':层函数]] \n
name为key

In [ ]:
class Net4(torch.nn.Module):
    def __init__(self):
        super(Net4,self).__init__()
        self.conv = torch.nn.Sequential(
            OrderedDict(
                [
                ("conv1",torch.nn.Conv2d(3,32,3,1,1)),
                ("relu1", torch.nn.ReLU()),
                ("pool", torch.nn.MaxPool2d(2))
                ]
            )
        )
        self.dense = torch.nn.Sequential(
            OrderedDict(
                [
                    ("dense1", torch.nn.Linear(32 * 3 * 3, 128)),
                    ("relu2", torch.nn.ReLU()),
                    ("dense2", torch.nn.Linear(128, 10))
                ]
            )
        )
    def forward(self,x):
        conv_out = self.conv1(x)
        res = conv_out.view(conv_out.size(0), -1)
        out = self.dense(res)
        return out